<a href="https://colab.research.google.com/github/Riccardolotorto/ripassoestivo2023_FlaskGeopandasPython/blob/main/RipassoEstivo_2023_geopandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas contextily

In [3]:
import geopandas as gpd
import contextily as ctx

In [4]:
province = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/ProvCM01012022_g")

In [5]:
comuni = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/Com01012022_g")

In [17]:
province3857 = province.to_crs(3857)
comuni3857 = comuni.to_crs(3857)

1. dato il nome di una provincia inserito dall'utente, visualizzarne i confini su una mappa di contextily

In [ ]:
provinciaUtente = input("Inserisci una provincia: ")
provinciaSelezionata = province3857[province3857["DEN_UTS"] == provinciaUtente.capitalize()]
ax = provinciaSelezionata.plot(figsize = (16, 8), edgecolor = "k", facecolor = "none")
ctx.add_basemap(ax)

2.  creare un nuovo geodataset contenente i comuni di quella provincia

In [18]:
comuniProvinciaSelezionata = comuni3857[comuni3857.within(provinciaSelezionata.geometry.item())]

3. visualizzare l'elenco dei comuni presenti in quella provincia. L'elenco deve essere in ordine alfabetico

In [ ]:
comuniProvinciaSelezionata[["COMUNE"]].sort_values(by="COMUNE").reset_index()

4.  costruire un dizionario contenente i nomi dei comuni della provincia inserita e la loro estensione

In [ ]:
comuni3857

In [ ]:
dizionario = dict(zip(comuniProvinciaSelezionata["COMUNE"], comuniProvinciaSelezionata["Shape_Area"]))
dizionario

5. progettare una funzione che trasformi una superficie da kmq in miglia quadrate

In [26]:
def convertitore(mq):
  kmq = mq / 1000000
  MigliaQ = kmq * 0.386102
  return MigliaQ

In [ ]:
convertitore(10)

6. applicare la funzione appena creata al geodataset dei comuni della provincia, aggiungendo una colonna denominata superficeInMigliaQuadrate

In [ ]:
comuniProvinciaSelezionata["superficeInMigliaQuadrate"] = convertitore(comuniProvinciaSelezionata["Shape_Area"])
comuniProvinciaSelezionata

7. creare una altro geodataset contenente i comuni che confinano con la provincia inserita dall'utente

In [29]:
comuniConfinantiProvinciaSelezionata = comuni3857[comuni3857.touches(provinciaSelezionata.geometry.item())]

8.  calcolare l'estensione complessiva  in miglia quadrate dei comuni che confinano con la provincia inserita dall'utente

In [ ]:
comuniConfinantiProvinciaSelezionata["superficeInMigliaQuadrate"] = convertitore(comuniConfinantiProvinciaSelezionata["Shape_Area"])
print(comuniConfinantiProvinciaSelezionata["superficeInMigliaQuadrate"].sum())

9.  visualizzare il comune confinante più esteso

In [32]:
comuneConfinanteMaggiore = comuniConfinantiProvinciaSelezionata[comuniConfinantiProvinciaSelezionata["Shape_Area"] == comuniConfinantiProvinciaSelezionata["Shape_Area"].max()][["COMUNE"]]

In [ ]:
comuneConfinanteMaggiore

10. calcolare la distanza tra il capoluogo della provincia inserita dall'utente e Milano

In [ ]:
distanza = comuni3857[comuni3857.COMUNE == "Lecco"].distance(comuni3857[comuni3857.COMUNE == "Milano"].geometry.item()) / 1000
distanza